# 캐글에서 데이터 로드

In [10]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'focusing-on-mobile-app-or-website:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F349053%2F688160%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240824%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240824T161635Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6baab5c83605873409a71233caffb856483b5d8df491bc5a13483369fb09c8c3f0104b3dec4ce81de82d6acc7134c0e1d0c743b5836f073fff1172c2d44fa7826d0a3d4921312f17dd11cb98e7f970cdc23a3fd891107b4c144c3eb9ffc368f600f0403272d73f7bf37e841d28eda59ae1eaa5c6dda8f46ec8ee254e30f469ea23a650a386e7eae456dd675808f544d58ea003854bef463b6a122f4e87cd91b94d020c5e71102041f64529d74029b42894f31877321a04aedd2b8b59ae53c7b3433e6b2ef6f8870f090a16073b3c165a8b4575d2e759ef21fff751e5688fb0714b2ac3a99a8b7938b9d1cdc4a321012896f4286b2499e0fc9a5d65475b5df430'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


������ ��θ� ã�� �� �����ϴ�.


OSError: [WinError 1314] 클라이언트가 필요한 권한을 가지고 있지 않습니다: '/kaggle/input' -> '..\\input'

<h2 style="font: bold 20px tahoma">
    문제 설명: 회사는 모바일 앱에 집중할지 아니면 웹사이트에 집중할지 결정하려고 한다.
</h2>

# 데이터 분석에 사용한 도구 가져오기

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.offline import iplot

from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score


# 데이터 세트 정보
#### ♣ Avatar: 고객이 선택한 아바타 색상을 나타내는 열

#### ♣ Avg. Session Length: 모바일 및 웹사이트의 평균 세션 시간(분)

#### ♣ Time on App: 고객이 모바일 앱 애플리케이션을 사용하는 데 소요한 총 시간(분)

#### ♣ Time on Website: 고객이 웹사이트에서 보낸 총 시간(분)

#### ♣ Length of Membership: 각 고객의 멤버십 또는 충성도 기간(월)

#### ♣ Yearly Amount Spent: 각 고객이 1년 동안 회사 제품에 지출한 총 금액

# 데이터 셋 로딩

In [ ]:
df = pd.read_csv("/kaggle/input/focusing-on-mobile-app-or-website/Ecommerce Customers")

# 데이터 셋 구성 요소 확인

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Email                 500 non-null    object 
 1   Address               500 non-null    object 
 2   Avatar                500 non-null    object 
 3   Avg. Session Length   500 non-null    float64
 4   Time on App           500 non-null    float64
 5   Time on Website       500 non-null    float64
 6   Length of Membership  500 non-null    float64
 7   Yearly Amount Spent   500 non-null    float64
dtypes: float64(5), object(3)
memory usage: 31.4+ KB


In [ ]:
df.sample(10, random_state=5)

,Email,Address,Avatar,Avg. Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
241,karenosborne@yahoo.com,"81814 Pratt Squares Suite 460\nNorth Robert, G...",SlateBlue,32.686245,12.638572,36.097221,4.297737,571.471034
448,flevine@gmail.com,5292 Melanie Crescent Apt. 064\nFischerborough...,AliceBlue,32.204655,12.480702,37.680288,3.279466,478.584286
75,langmatthew@hotmail.com,"606 Perez Drives\nMaryside, CO 94387-5877",DimGray,32.049839,12.238057,38.730862,3.120569,478.719357
212,baldwinbryan@estrada-silva.biz,"1470 Kathleen Pass\nSouth Christopherberg, SD ...",MediumSeaGreen,33.304431,12.378490,38.764297,3.843849,536.130897
481,autumn88@mendoza-mills.com,"214 Obrien Lakes Suite 572\nSouth Jeremy, KS 5...",MediumOrchid,32.047815,12.482670,35.536025,3.393903,497.389558
280,carrillojacob@perry-larsen.com,"909 Hicks Mountains\nGabriellaport, MD 33121",Aquamarine,32.271848,13.485009,37.550880,3.086337,511.979860
474,antonioharris@hotmail.com,"4307 Nicholas Drive Apt. 259\nRamirezberg, AS ...",MediumBlue,33.700886,13.471578,37.071643,2.379076,492.556834
269,sharper@yahoo.com,"239 Bush Fall Apt. 906\nRiveraville, KY 88906-...",SandyBrown,34.318927,13.402332,37.292045,3.606087,585.931844
40,rhonda96@little.org,"2977 Perez Row\nLake Jack, TN 45336-9436",GhostWhite,32.070546,11.733106,37.534291,4.671275,532.751788
222,sanchezkara@hotmail.com,"40494 Robert Park\nHeatherside, IL 70364",SlateBlue,34.334865,11.109456,38.585855,3.892891,502.409785


<h3 style = "font: bold 18px arial;
             color: gold;
             background-color: #111;
             padding: 20px;
             border: 4px solid orangered;
             border-radius: 8px">
    # 먼저 데이터를 살펴보면, 사용자들은 모바일 앱보다 웹사이트에서 더 많은 시간을 보내는 것을 알 수 있다.
    <br>
    <br>
    ► 그러면 사용자가 웹사이트에서 더 많은 시간을 보낼수록 일년 기준 더 많은 돈을 지출한다고 말할 수 있을까?
</h3>

# 데이터 살펴보기

In [ ]:
# 데이터 카테고리 설명
df.select_dtypes(include="object").describe()

,Email,Address,Avatar
count,500,500,500
unique,500,500,138
top,mstephenson@fernandez.com,"835 Frank Tunnel\nWrightmouth, MI 82180-9605",SlateBlue
freq,1,1,7


In [ ]:
# 정량 데이터 설명
np.round(df.describe().T, 2)

,count,mean,std,min,25%,50%,75%,max
Avg. Session Length,500.0,33.05,0.99,29.53,32.34,33.08,33.71,36.14
Time on App,500.0,12.05,0.99,8.51,11.39,11.98,12.75,15.13
Time on Website,500.0,37.06,1.01,33.91,36.35,37.07,37.72,40.01
Length of Membership,500.0,3.53,1.00,0.27,2.93,3.53,4.13,6.92
Yearly Amount Spent,500.0,499.31,79.31,256.67,445.04,498.89,549.31,765.52


In [ ]:
# Clean The Columns' Name from Any Spaces
df.columns = df.columns.str.replace(" ",  "_").str.replace(".", "")

In [ ]:
df.rename(columns={"Time_on_App": "App_Usage",
                   "Time_on_Website": "Website_Usage",
                  "Length_of_Membership":"Membership_Length",
                  "Yearly_Amount_Spent": "Yearly_Spent"}, inplace=True)

In [ ]:
df.head()

,Email,Address,Avatar,Avg_Session_Length,App_Usage,Website_Usage,Membership_Length,Yearly_Spent
0,mstephenson@fernandez.com,"835 Frank Tunnel\nWrightmouth, MI 82180-9605",Violet,34.497268,12.655651,39.577668,4.082621,587.951054
1,hduke@hotmail.com,"4547 Archer Common\nDiazchester, CA 06566-8576",DarkGreen,31.926272,11.109461,37.268959,2.664034,392.204933
2,pallen@yahoo.com,"24645 Valerie Unions Suite 582\nCobbborough, D...",Bisque,33.000915,11.330278,37.110597,4.104543,487.547505
3,riverarebecca@gmail.com,"1414 David Throughway\nPort Jason, OH 22070-1220",SaddleBrown,34.305557,13.717514,36.721283,3.120179,581.852344
4,mstephens@davidson-herman.com,"14023 Rodriguez Passage\nPort Jacobville, PR 3...",MediumAquaMarine,33.330673,12.795189,37.536653,4.446308,599.406092


# Correlation HeatMap & Charts

In [ ]:
correlation = df.corr(numeric_only=True)

fig = px.imshow(
    correlation,
    template = "plotly_dark",
    text_auto = "0.2f",
    aspect=1,
    color_continuous_scale="orrd",
    title= "Correlations Between Data"
)
fig.update_traces(
    textfont = {
        "size" : 16,
        "family" :"consolas"
    }

)
fig.update_layout(
    title = {
        "font" :{
            "size" : 28,
            "family" : "tahoma"
        }
    }
)
iplot(fig)

In [ ]:
fig = px.scatter_matrix(
    df,
    dimensions=df.select_dtypes(include="number").columns,
    height=950,
    color="Yearly_Spent",
    opacity=0.65,
    title= "Relationships Between Numerical Data",
    template="plotly_dark"

)

fig.update_layout(
    title = {
        "font" :{
            "size" : 28,
            "family" : "tahoma"
        }
    }
)
iplot(fig)

<h3 style = "font: bold 18px arial;
             color: gold;
             background-color: #111;
             padding: 15px;
             border: 2px solid orangere">
    ► 인사이트 정리
    <br>
    <br>
    • Yearly_Spent(연간 지출)와 Membership_Length(멤버십 기간) 사이에는 선형의 <b style = "color: tomato">강한</b> 양의 상관관계가 있다.
    <br>
    <br>
    • Yearly_Spent(연간 지출)과 App Usage Time(앱 사용 시간)에는<b style = "color: lightgreen">중간 정도</b> 선형의 양의 상관관계가 있다.
    <br>
    <br>
    • Yearly_Spent(연간 지출)과 Avg_Session_Length(모바일 및 웹사이트의 평균 세션 시간(분))에는 선형의 <b style = "color: lightblue">약한</b> 양의 상관관계가 있다.
     <br>
    <br>
    • Yearly_Spent(연간 지출)과 Website Usage Time(웹사이트 사용 시간) 사이에는 거의 상관관계가 <b style = "color: #ccc">없다</b>.
</h3>

# 다중 선형 회귀 방정식


# 상관관계 != 인과관계


*   상관관계 : 상관관계(Correlation) 는 두 변수 간의 연관성을 나타내는 통계적 개념입니다. 이는 두 변수 사이의 관계를 측정하는 방법으로, 한 변수가 변할 때 다른 변수가 어떻게 변하는지를 나타냅니다. 상관관계의 정도는 상관계수(correlation coefficient)로 표현되며, 이 값은 -1에서 1 사이의 값을 가집니다. 상관계수의 크기와 방향에 따라 상관관계의 강도와 유형을 알 수 있습니다.

양의 상관관계 (Positive Correlation)
양의 상관관계는 두 변수가 같은 방향으로 변할 때 나타납니다. 상관계수가 +1에 가까울수록 강한 양의 상관관계를 나타냅니다.

예시

아이스크림 판매량과 기온: 기온이 올라가면 아이스크림 판매량도 증가합니다.
키와 몸무게: 일반적으로 키가 큰 사람일수록 몸무게도 많이 나갑니다.
음의 상관관계 (Negative Correlation)
음의 상관관계는 두 변수가 반대 방향으로 변할 때 나타납니다. 상관계수가 -1에 가까울수록 강한 음의 상관관계를 나타냅니다.

예시

기온과 난방비 : 기온이 낮아질수록 난방비가 증가합니다.
자동차 주행 거리와 연료 잔량 : 주행 거리가 증가할수록 연료 잔량은 감소합니다.
무상관관계 (No Correlation)
무상관관계는 두 변수 간에 아무런 연관성이 없을 때 나타납니다. 상관계수가 0에 가까울수록 무상관관계를 나타냅니다.

예시:

신발 사이즈와 시험 점수: 두 변수 간에는 아무런 연관성이 없습니다.
출생 월과 지능 지수: 출생 월이 지능 지수에 영향을 미치지 않습니다.


*   인과관계 : 인과관계(Causation) 는 한 사건(원인)이 다른 사건(결과)을 직접적으로 유발하는 관계를 의미합니다.

인과관계에서 중요한 점은 시간적 선후관계와 배타적 인과관계입니다. 즉, 원인이 먼저 발생하고, 그에 따라 결과가 뒤따라야 하며, 두 변수 간의 관계가 다른 변수에 의해 설명되지 않아야 합니다.
=> 회귀분석

# 변수 선택

*   Y = 원인 = 독립 변수
*   X = 결과 = 종속 변수



In [ ]:
X = df[['Avg_Session_Length', 'App_Usage', 'Website_Usage', 'Membership_Length']]
y = df["Yearly_Spent"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=38)

In [ ]:
X_train.head()

,Avg_Session_Length,App_Usage,Website_Usage,Membership_Length
482,30.971676,11.731364,36.074551,4.426364
260,35.039283,14.426491,37.374184,3.930615
46,34.564558,13.146551,37.335446,3.876875
334,31.976480,10.757131,36.595868,1.977007
469,31.169507,13.970181,36.673953,1.785174


# 회귀 분석 모델 만들기

In [ ]:
model = linear_model.LinearRegression()

In [ ]:
kf = KFold(n_splits=25, shuffle=True, random_state=99)
scores = cross_val_score(model, X, y, cv=kf)
print(f"Model Score: {np.mean(scores) * 100:.2f}%")

Model Score: 98.25%


In [ ]:
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
train_score = model.score(X_train, y_train)
print(f"Train Score: {train_score * 100:.2f}%")

Train Score: 98.46%


# 데이터 예측 하기

In [ ]:
predections = model.predict(X_test)

In [ ]:
test_score = r2_score(y_test, predections)
print(f"Test Score: {test_score * 100:.2f}%")

Test Score: 98.27%


In [ ]:
error_ratio = mean_absolute_error(y_test, predections)
print(f"Error Ratio: {error_ratio:0.2f}")

Error Ratio: 7.71


In [ ]:
d = {
    "Actual_Yearly_Spent": y_test,
    "Predicted_Yearly_Spent": predections
}
predected_df = pd.DataFrame(d)

predected_df.head()

,Actual_Yearly_Spent,Predicted_Yearly_Spent
446,529.194519,531.190970
147,479.731938,478.287472
314,610.128033,604.291007
414,532.724805,546.204383
476,408.958336,412.507458


In [ ]:
fig = px.scatter(
    predected_df,
    x = "Predicted_Yearly_Spent",
    y = "Actual_Yearly_Spent",
    color = predected_df["Predicted_Yearly_Spent"] - predected_df["Actual_Yearly_Spent"],
    opacity=0.8,
    title= "Predicted Vs. Actual",
    template="plotly_dark",
    trendline="ols"

)

fig.update_layout(
    title = {
        "font" :{
            "size" : 28,
            "family" : "tahoma"
        }
    }
)
iplot(fig)

In [ ]:
# pd.to_pickle(model, "linear_regression_yearly_spent_predictor_v1.pkl")

# 모델 성능

<h3 style = "font: bold 20px arial;
             color: gold;
             background-color: #111;
             padding: 15px;
             border: 2px solid orangere">
    • Cross Validation Score: <b style = "color: lightgreen">98.25% </b>
    <br>
    <br>
    • Train Score: <b style = "color: lightgreen">98.46%</b>
    <br>
    <br>
    • Test Score: <b style = "color: lightgreen">98.27% </b>
</h3>

In [ ]:
theta = model.coef_.T
print("Independent Feature\tCoefficient".expandtabs(25))
print("="*36)
for i in range(X_train.shape[1]):
    print(f"{X_train.columns[i]}\t{theta[i]:0.3f}".expandtabs(25))

Independent Feature      Coefficient
Avg_Session_Length       25.454
App_Usage                38.791
Website_Usage            0.220
Membership_Length        61.490


# 데이터 분석 결론


*   사용자 연간 지출에 가장 큰 영향을 미치는 변수는 모바일 앱 사용 시간과 멤버십 기간이다.
*   초반에 예상했던 것과는 달리 웹사이트에 머무는 시간은 거의 영향을 미치지 않는다는 것을 알 수 있다.
